# Iterative model building

This Notebook contains functions to perform iterative model building of pHMMs from an input protein query against a target dataset.
The queries in this Notebook are for the 4 "*core*" indole diterpene proteins from the *Penicillium paxilli* paxilline BGC. 
These are obtained from the curated SWISSPROT database, against which these queries are searched.

In [1]:
import gzip
import urllib.request
import re
import pyhmmer
import matplotlib.pyplot as plt
from math import ceil
from math import floor

ModuleNotFoundError: No module named 'pyhmmer'

In [2]:
def build_seqreader(multifasta):
    """
    Opens multiFASTA using pyHMMER and converts it to binary
    'loaded' format.
    """
    if multifasta.endswith(".gz"):
        reader = gzip.open(multifasta, "rb")
    else:
        reader = open(multifasta, "r")
        
    with pyhmmer.easel.SequenceFile(reader, digital=True, alphabet=alphabet) as seq_file:
        dataset = seq_file.read_block()
    
    return dataset

def map_taxonomy(dataset):
    """
    Parse the taxonomy from a loaded pyHMMER dataset.
    """

    taxonomy_by_name = {}
    organism_by_name = {}
    description_by_name = {}

    for seq in dataset:
        match = re.search(b"(.*) OS=(.*) OX=(\d+)", seq.description)
        taxonomy_by_name[seq.name] = int(match.group(3))
        organism_by_name[seq.name] = match.group(2)
        description_by_name[seq.name] = match.group(1)

def model_building(dataset, query):
    """
    Completes the pyHMMER hmmsearch pipeline on a loaded dataset.
    """
    alphabet = pyhmmer.easel.Alphabet.amino()
    pli = pyhmmer.plan7.Pipeline(alphabet)
    hits = pli.search_seq(query, dataset)
    
    plt.legend(frameon=False)
    plt.plot([hit.score for hit in hits], "o-")
    plt.xlabel("Hit rank")
    plt.ylabel("Score (bits)")
    plt.savefig("uniprot.svg", format="svg")
    plt.show()
    
def iterative_model_building(dataset, query, bitscore_cutoff):
    """
    Performs iterative hmmsearch against loaded dataset using a
    specific trusted bitscore cutoff.    
    """
    alphabet = pyhmmer.easel.Alphabet.amino()

    pli = pyhmmer.plan7.Pipeline(alphabet, incT=int(bitscore_cutoff), incdomT=int(bitscore_cutoff))
    search = pli.iterate_seq(query, dataset)
    
    iterations = []
    while not search.converged:
        iteration = next(search)
        iterations.append(iteration)
        print(f"Iteration={iteration.iteration} Hits={len(iteration.hits):2} Included={iteration.hits[0].name} Converged={search.converged}")

    return iterations

def plot_iterations(iterations, out_filename):
   """
   Prepares lineplots of iteration hit trusted bitscores.
   """
    for iteration in iterations:
        plt.plot([hit.score for hit in iteration.hits], "o-", label=f"Iteration {iteration.iteration}")

    plt.legend(frameon=False)
    plt.xlabel("Hit rank")
    plt.ylabel("Score (bits)")
    plt.savefig(out_filename, format="svg")
    plt.show()

def print_iteration_diffs(iterations):
    """ 
    Describes the hits included in a pHMM model following iteratvie search if it
    is above the trusted bitscore.
    """
    for hit in iterations[-1].hits:
        if hit.included:
            print(f"{hit.name.decode():<20}\t{hit.score:5.1f}\t{description_by_name[hit.name].decode():34}\t{organism_by_name[hit.name][:60].decode()}")
            
def save_hmm(hmm, name):
    """ 
    Save pHMM to a H3MM file.
    """
    byte_obj = name.encode('ascii')
    hmm.name = byte_obj
    hmm.description = None
    hmm.accession = None
    with open(f"{name}.hmm", "wb") as dst_file:
        hmm.write(dst_file)

def get_cutoffs(iterations):
    """ 
    Calculate all bitscore cutoffs for a pyhhmmer iterative search.
    """

    hmm = iterations[-1].hmm
    # noise
    noise_score = max(hit.score for hit in iterations[-1].hits if not hit.included)
    noise_domscore = max(hit.best_domain.score for hit in iterations[-1].hits if not hit.included)
    hmm.cutoffs.noise = ceil(noise_score), ceil(noise_domscore)
    print(f"Noise: {hmm.cutoffs.noise}")
    # gathering
    gathering_score = min(hit.score for hit in iterations[-1].hits if hit.included)
    gathering_domscore = min(hit.best_domain.score for hit in iterations[-1].hits if hit.included)
    hmm.cutoffs.gathering = floor(gathering_score), floor(gathering_domscore)
    print(f"Gathering: {hmm.cutoffs.gathering}")
    # trusted
    i = max(i for i, hit in enumerate(iterations[-1].hits) if hit.score > 350)
    hmm.cutoffs.trusted = floor(iterations[-1].hits[i].score), floor(iterations[-1].hits[i].best_domain.score)
    print(f"Trusted: {hmm.cutoffs.trusted}")
    
def main(query, data, min_bitscore, output):

    # load SwissProt database
    dataset = build_seqreader("uniprot_sprot.fasta.gz")
    map_taxonomy(dataset)
    
    # perform iterative query search
    model_building(dataset, query)    
    iterations = iterative_model_building(dataset, query, min_bitscore)
    plot_iterations(iterations, "SwissProt_iterations.svg")
    print_iteration_diffs(iterations)
    get_cutoffs(iterations)
    hmm = iterations[-1].hmm
    save_hmm(hmm, output)

Download the Swissprot database and open it with pyhmmer:

In [3]:
alphabet = pyhmmer.easel.Alphabet.amino()

url = "http://ftp.ebi.ac.uk/pub/databases/uniprot/knowledgebase/uniprot_sprot.fasta.gz"
with urllib.request.urlopen(url) as response:
    reader = gzip.open(response, "rb")
    with pyhmmer.easel.SequenceFile(reader, digital=True, alphabet=alphabet) as seq_file:
        swissprot = seq_file.read_block()

NameError: name 'pyhmmer' is not defined

Build iterative models of each "core" IDT protein with the trusted cutoff 
bitscores used.

In [4]:
paxb = next(x for x in swissprot if b'E3UBL6' in x.name)
paxc = next(x for x in swissprot if b'Q9C448' in x.name)
paxg = next(x for x in swissprot if b'Q9C446' in x.name)
paxm = next(x for x in swissprot if b'Q9C447' in x.name)
main(paxb, swissprot, 400, "PAXB_swissprot.phmm")
main(paxc, swissprot, 350, "PAXB_swissprot.phmm")
main(paxg, swissprot, 460, "PAXB_swissprot.phmm")
main(paxm, swissprot, 500, "PAXB_swissprot.phmm")

NameError: name 'dataset' is not defined